In [6]:
import tensorflow as tf
import os

In [12]:
with tf.name_scope("Train"):

#########################PREPARE DATA##################################################
    b = tf.Variable(0., name="bias")
    w = tf.Variable(tf.zeros([5,1]), name="weights")
        
    def read_csv(batch_size, file_name, record_defaults):
        filename_queue = tf.train.string_input_producer([os.path.dirname("__file__") + "/" + file_name])
        
        reader = tf.TextLineReader(skip_header_lines=1)
        key, value = reader.read(filename_queue)
        
        #Creating a tensor from type string in a tuple of tensor columns with a specified
        #defaults, which also sets the data type for each column
        decoded = tf.decode_csv(value, record_defaults=record_defaults)
        
        #batch actually reads the file and loads "batch_size" rows in a single tensor
        return tf.train.shuffle_batch(decoded, batch_size=batch_size,
                                     capacity = batch_size * 50,
                                     min_after_dequeue=batch_size)
    
        
        
    def inputs():
        #Get values from csv(coma separated values)
        pasenger_id, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked = \
            read_csv(100, "train.csv", [[0.0], [0.0], [0], [""], [""], [0.0], [0.0], [0.0], [""], [0.0], [""], [""]])
    
        #Convert it to categorical data
        is_first_class = tf.to_float(tf.equal(pclass, [1]))
        is_second_class = tf.to_float(tf.equal(pclass, [2]))
        is_third_class = tf.to_float(tf.equal(pclass, [3]))
        
        gender = tf.to_float(tf.equal(sex, ["female"]))
        
        #Finally we pack all the fdeatures in a single matrix
        #We then transpose to have a matrix with one example per row and one feature per column. 
        features = tf.transpose(tf.pack([is_first_class, is_second_class, is_third_class, gender, age]))
        survived = tf.reshape(survived, [100,1])
        
        return features, survived
        
###############################GET ON MATHS#################################################
    def combine_inputs(x):
        return tf.matmul(x, w) + b
    
    def inference (x):
        ## do maths with x definning the kind of problem you want to resolve
        ## Logistic regression = Log( linear_regression )
        return tf.sigmoid(combine_inputs(x))
    
    def loss (x, y):
        ##loss = sum(y*log(y_predict) + (1 - y ) * log( 1 - y_predict))
        y_predicted = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(combine_inputs(x), y))
        return y_predicted
    
    def train (total_loss):
        #Gradient Descent
        learning_rate = 0.01
        return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss) 
        
    #def evaluate (sess, x, y):
        #After training, evaluate results
        #predicted = tf.cast(inference(x) > 0.5, tf.float32)
        #print sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, y), tf.float32)))

In [14]:
with tf.Session() as sess:
    
    tf.initialize_all_variables().run()
    
    with tf.name_scope("Placeholders"):
        x, y = inputs()
        
    total_loss = loss(x,y)
    train_operation = train(total_loss)
     
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess, coord= coord)

    train_cicle = 10000
    
    for i in xrange(train_cicle):
        sess.run(train_operation)
        
        if(i % 100 == 0):
            print (total_loss)
            
    ##evaluation
    
    coord.request_stop()
    coord.join(thread)
    
    sess.close()
    

OutOfRangeError: RandomShuffleQueue '_16_Placeholders_5/shuffle_batch/random_shuffle_queue' is closed and has insufficient elements (requested 100, current size 0)
	 [[Node: Placeholders_5/shuffle_batch = QueueDequeueMany[_class=["loc:@Placeholders_5/shuffle_batch/random_shuffle_queue"], component_types=[DT_FLOAT, DT_FLOAT, DT_INT32, DT_STRING, DT_STRING, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_STRING, DT_FLOAT, DT_STRING, DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](Placeholders_5/shuffle_batch/random_shuffle_queue, Placeholders_5/shuffle_batch/n)]]

Caused by op u'Placeholders_5/shuffle_batch', defined at:
  File "/home/empanada/anaconda/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/empanada/anaconda/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/empanada/anaconda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3,